In [ ]:
import pickle
import numpy as np


documents = pickle.load(open("FakeNewsCorpus0/dict_documents.pkl", "rb"))
indexes = np.load('FakeNewsCorpus0/indexes.npy')
lbls = np.load('FakeNewsCorpus0/labels.npy')

texts1 = []
labels1 = []
for i in range(len(indexes)):
    index = indexes[i]
    texts1.append(documents[index])
    if lbls[i] == np.str_("-1"):
      labels1.append(0)
    else :
      labels1.append(1)

print("lable 1: " + str(labels1[0]))
print("text 1: " + texts1[0])

lable 1: 1
text 1: Veteran Commentator Calls Out the Growing Ethnonationalism at Fox News (and Its SO Ugly)  % of readers think this story is Fact. Add your two cents.  Headline: Bitcoin & Blockchain Searches Exceed Trump! Blockchain Stocks Are Next!  Lets be honest: This is pretty much all of Fox News commentary portion now, since the network became Trump Pravda.  Commentator and The Weekly Standard founder, Bill Kristol, sat for an interview with CNBC that was released on Thursday. Among various topics, he noted how Tucker Carlson, a Fox News superstar, has changed since he began with The Weekly Standard.  Speaking of Carlsons Fox News show:  I mean, it is close now to racism, white  I mean, I dont know if its racism exactly  but ethnonationalism of some kind, lets call it. A combination of dumbing down, as you said earlier, and stirring peoples emotions in a very unhealthy way, Kristol told CNBCs John Harwood in the interview published on Thursday.  Carlson has been getting a bit of

In [ ]:
documents = pickle.load(open("FakeNewsCorpus1/dict_documents.pkl", "rb"))
indexes = np.load('FakeNewsCorpus1/indexes.npy')
lbls = np.load('FakeNewsCorpus1/labels.npy')

texts2 = []
labels2 = []
for i in range(len(indexes)):
    index = indexes[i]
    texts2.append(documents[index])
    if lbls[i] == np.str_("-1"):
      labels2.append(0)
    else :
      labels2.append(1)

print("lable 1: " + str(labels2[0]))
print("text 1: " + texts2[0])

lable 1: 0
text 1: (Photo: Reuters/Jean-Paul Pelissier) Visitors look at "Piss Christ," a piece of art by U.S. artist Andres Serrano, partially destroyed by catholic activists in Avignon, April 19, 2011. The Piss Christ created in 1987, is a photography representing a small plastic crucifix submerged in a glass of the artist's urine.  The Associated Press removed a photo of an art piece called "Piss Christ" from its website Wednesday after a journalist pointed out the company's double standard. AP has refused to publish the Charlie Hebdo cartoons mocking Muhammad that led to Hebdo's murder due to its policy of not publishing "deliberately provocative images."  Expand | Collapse (Photo: Twitter) French magazine Charlie Hebdo's September cover is seen in this image.  In a Wednesday interview with The Daily Beast, an AP spokesperson explained why the company did not publish, or cropped, images of Charlie Hebdo cartoons.  "It's been our policy for years that we refrain from moving delibera

In [ ]:
texts = [*texts1, *texts2]
labels = [*labels1, *labels2]

In [ ]:
count = 0
for text in texts:
  if len(text.split(' ')) > 512:
    count += 1
count

8692

In [ ]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=.2)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [ ]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
MAX_TRAIN_LENTH = 400

In [ ]:
train_encodings = tokenizer(train_texts, padding='max_length', truncation=True, max_length=MAX_TRAIN_LENTH)
val_encodings = tokenizer(val_texts, padding='max_length', truncation=True, max_length=MAX_TRAIN_LENTH)
test_encodings = tokenizer(test_texts, padding='max_length', truncation=True, max_length=MAX_TRAIN_LENTH)

In [ ]:
import torch

class FNNDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = FNNDataset(train_encodings, train_labels)
val_dataset = FNNDataset(val_encodings, val_labels)
test_dataset = FNNDataset(test_encodings, test_labels)

In [ ]:
# Checks if there is a gpu available
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU: {}'.format(torch.cuda.get_device_name(0)))
else:
    print('GPU device not found')

Found GPU: Tesla T4


In [ ]:
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model.to(device)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
from datasets import load_metric

def compute_metrics(model, eval_dataloader, metric):
  model.eval()
  for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

  return metric.compute()

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16)
test_dataloader = DataLoader(test_dataset, batch_size=16)
optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(10):
  model.train()
  for batch in train_loader:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs[0]
    loss.backward()
    optim.step()
  print('eval accuracy results for epoch ' + str(epoch) + ': ' + str(compute_metrics(model, val_dataloader, load_metric('accuracy'))['accuracy']))
  print('')

model.eval()

print('trainig has finished')
print('-------------------------------------------------------------------')
print('final evaluation is: ')

whantedStatistics = ['accuracy', 'precision', 'recall', 'f1']
for statistic in whantedStatistics:
  metric = load_metric(statistic)
  print(statistic + ': ' + str(compute_metrics(model, test_dataloader, metric)[statistic]))

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


eval accuracy results for epoch 0: 0.9807291666666667

eval accuracy results for epoch 1: 0.9890625

eval accuracy results for epoch 2: 0.9916666666666667

eval accuracy results for epoch 3: 0.990625

eval accuracy results for epoch 4: 0.9911458333333333

eval accuracy results for epoch 5: 0.9875

eval accuracy results for epoch 6: 0.9859375

eval accuracy results for epoch 7: 0.9885416666666667

eval accuracy results for epoch 8: 0.9890625

eval accuracy results for epoch 9: 0.9916666666666667

trainig has finished
-------------------------------------------------------------------
final evaluation is: 
accuracy: 0.9916666666666667


precision: 0.9983498349834984


recall: 0.9853420195439739


f1: 0.9918032786885247


In [ ]:
tokenizer.save_pretrained("fake-news-heatmap-creator")
model.save_pretrained("fake-news-heatmap-creator")

In [ ]:
!python -m transformers.onnx --model=fake-news-heatmap-creator onnx/

Some weights of the model checkpoint at fake-news-heatmap-creator were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using framework PyTorch: 1.10.0+cu111
Overriding 1 configuration item(s)
	- use_cache -> False
Validating ONNX model...
	-[✓] ONNX model output names match reference model ({'last_hidden_state'})
	- Validating ONNX Model output "last_hidden_state":
		-[✓] (2, 8, 768) matches (2, 8, 768)
		-[✓] all values close (atol: 1e-05)
All good, model saved at: onnx/model.onnx


In [ ]:
!zip -r /content/fake-news-heatmap-creator /content/fake-news-heatmap-creator

  adding: content/fake-news-heatmap-creator/ (stored 0%)
  adding: content/fake-news-heatmap-creator/tokenizer_config.json (deflated 39%)
  adding: content/fake-news-heatmap-creator/tokenizer.json (deflated 71%)
  adding: content/fake-news-heatmap-creator/vocab.txt (deflated 53%)
  adding: content/fake-news-heatmap-creator/pytorch_model.bin (deflated 7%)
  adding: content/fake-news-heatmap-creator/special_tokens_map.json (deflated 40%)
  adding: content/fake-news-heatmap-creator/config.json (deflated 49%)


In [ ]:
!zip -r /content/onnx /content/onnx

  adding: content/onnx/ (stored 0%)
  adding: content/onnx/model.onnx (deflated 7%)
